# Домашнее задание 1. Линейные модели

### Хрулева Елена ИУ8-83
### Вариант 19

## Наборы данных

### Задача регрессии

D. Набор данных Таблица insurance.csv. Задача: построить модель, предсказывающую стоимость медицинской страховки (колонка charges). Baseline: RMSE (root mean square error) = 5800

### Задача классификации

N. Набор данных Таблица diabetes.csv. Задача: детектировать, есть ли у человека диабет. Baseline: sklearn.metrics.f1_score(average='macro') = 0.73


## 2. Задача классификации (9 баллов)

Постройте модель машинного обучения, решающую задачу классификации на вашем наборе данных. Задание считается выполненным, если соответствующая оценка качества на тестовой выборке <= baseline.

### 0. Инициализация ноутбука

Загрузка необходимых библиотек

In [163]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from numpy import arange
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

baseline = 0.73

### 2.1 Загрузка данных

Загрузите данные в датафрейм при помощи функции read_csv блиблиотеки pandas

In [164]:
data = pd.read_csv(r'C:\Users\alenk\Downloads\diabetes.csv')

data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### 2.2 Разделение набора данных на тренировочный и тестовый

Разделите набор данных на тренировочный и тестовый при помощи функции train_test_split библиотеки ???

In [165]:
correctData = pd.get_dummies(data) # визуально все данные являются числовыми, но лучше перестраховаться
withoutTeacher = correctData.loc[:, correctData.columns != 'Outcome']
teacher = correctData['Outcome']

withoutTeacherTraining, withoutTeacherTest, teacherTraining, teacherTest = train_test_split(withoutTeacher, teacher, train_size=0.80, random_state=4)

### 2.3 Предобработка данных

Проведите необходимую предобработку (перенормировку, выделение главных компонент и т.д.) данных. Приведите данные к такому виду, чтобы их можно было использовать для обучения моделей.

In [166]:
# Перенормировка
scalerMinMax = MinMaxScaler() 
normalizedTraining = scalerMinMax.fit_transform(withoutTeacherTraining) 
normalizedTest = scalerMinMax.transform(withoutTeacherTest) 

# Выделение главных компонент
pca = PCA(n_components=0.9) 
pca_Training = pca.fit_transform(normalizedTraining)
pca_Test = pca.transform(normalizedTest)

In [167]:
normalizedTraining.shape

(614, 8)

In [168]:
normalizedTest.shape

(154, 8)

In [169]:
pca_Training.shape

(614, 6)

In [170]:
pca_Test.shape

(154, 6)

### 2.4 Логистическая регрессия

Обучите модель логистической регрессии, используя стандартную реализацию sklearn.linear_model.LogisticRegression. Оцените метрики качества полученной модели при помощи sklearn.metrics.classification_report и sklearn.metrics.f1_score.

In [171]:
def ScoreChecking(score):
    if score > baseline:
        raise Exception(f'sklearn.metrics.f1_score({score}) > baseline ({baseline})')
    print(f'sklearn.metrics.f1_score = {score} and <= baseline = {baseline}')

logisticRegression = LogisticRegression()
logisticRegression.fit(pca_Training, teacherTraining)
prediction = logisticRegression.predict(pca_Test)

report = classification_report(prediction, teacherTest)

print(f'Отчет при помощи sklearn.metrics.classification_report:\n\n{report}\n')

score = f1_score(prediction, teacherTest, average='macro')

ScoreChecking(score)

Отчет при помощи sklearn.metrics.classification_report:

              precision    recall  f1-score   support

           0       0.89      0.78      0.83       116
           1       0.52      0.71      0.60        38

    accuracy                           0.77       154
   macro avg       0.71      0.75      0.72       154
weighted avg       0.80      0.77      0.78       154


sklearn.metrics.f1_score = 0.7174311926605506 and <= baseline = 0.73


### 2.5 Подбор гиперпараметра регуляризации

При помощи sklearn.model_selection.GridSearchCV переберите оптимальные параметры C в диапазоне 0.01 до 1 с шагом 0.1.

In [174]:
gridSearch=GridSearchCV(LogisticRegression(),{'C': arange(0.01,1.01,0.1)})
gridSearch.fit(pca_Training, teacherTraining)


print(f'Лучшая модель: {gridSearch.best_estimator_}\n')
print(f'Ошибка, полученная на лучшей модели: {gridSearch.best_score_}\n')
print(f'Гиперпараметры лучшей модели: {gridSearch.best_params_}\n')

Лучшая модель: LogisticRegression(C=0.81)

Ошибка, полученная на лучшей модели: 0.7279888044782087

Гиперпараметры лучшей модели: {'C': 0.81}



### 2.6 Выводы

Сравните качество (величину ошибки) для моделей из пп. 2.4-2.5. Какая модель показала наилучший результат?



In [173]:
logisticRegression = LogisticRegression(C=0.81)
logisticRegression.fit(pca_Training, teacherTraining)
prediction = logisticRegression.predict(pca_Test)

report = classification_report(prediction, teacherTest)

print(f'Отчет при помощи sklearn.metrics.classification_report:\n\n{report}\n')

score = f1_score(prediction, teacherTest, average='macro')

ScoreChecking(score)

Отчет при помощи sklearn.metrics.classification_report:

              precision    recall  f1-score   support

           0       0.89      0.78      0.83       116
           1       0.52      0.71      0.60        38

    accuracy                           0.77       154
   macro avg       0.71      0.75      0.72       154
weighted avg       0.80      0.77      0.78       154


sklearn.metrics.f1_score = 0.7174311926605506 and <= baseline = 0.73


LogisticRegression при C=0.81 и LogisticRegression без указания параметра С дали одинаковый результат